DSCI Final Project - Baseball Statistics

MLB Stats API: <br>
https://pypi.org/project/MLB-StatsAPI/ <br>
Wiki/Documentation: https://github.com/toddrob99/MLB-StatsAPI/wiki <br>
Usage: run 'pip install MLB-StatsAPI' <br><br>
Datasets:<br>
SFBB-Player-ID-Map.csv: from https://www.smartfantasybaseball.com/tools/ <br>
expected_stats.csv: from https://baseballsavant.mlb.com/leaderboard/expected_statistics?type=batter&year=2015&position=&team=&min=q <br>
Batting.csv: from https://www.kaggle.com/datasets/open-source-sports/baseball-databank?select=Batting.csv

This file, written by Andrew, deals largely with comparisons between left and right handed players (Question 1/2)

In [11]:
# Setups
import pandas as pd
import plotly.express as px
import statsapi
#import logging

In [12]:
data_location = "datasets/"

exp_stats = pd.read_csv(data_location + "expected_stats.csv")
batting_names = pd.read_csv(data_location + 'SFBB-Player-ID-Map.csv')
batting = pd.read_csv(data_location + 'Batting.csv')

In [13]:
#renames columns of exp_stats and makes it easier to view
exp_stats = exp_stats.rename(columns={'last_name':'Name',
                                      ' first_name':'to_drop',
                                      'player_id':'ID',
                                      'year':'Year',
                                      'pa':'Plate Appearances',
                                      'bip':'Balls In Play',
                                      'ba':'BA',
                                      'est_ba':'xBA',
                                      'est_ba_minus_ba_diff':'xBA - BA Diff',
                                      'slg':'Slg',
                                      'est_slg':'xSlg',
                                      'est_slg_minus_slg_diff':'xSlg - Slg Diff',
                                      'woba':'WOBA',
                                      'est_woba':'xWOBA',
                                      'est_woba_minus_woba_diff':'xWOBA - WOBA Diff'})
exp_stats['Name'] = exp_stats['to_drop'] + ' ' + exp_stats['Name']
exp_stats = exp_stats.drop(columns =['to_drop','Year'])
#exp_stats = exp_stats.set_index('Name')
exp_stats = exp_stats.replace(r"^ +| +$", r"", regex=True)

In [14]:
#renames columns of batting and makes it easier to view
batting = batting[batting.yearID >= 2010]
batting = batting.drop(columns=['stint', 'lgID'])
batting = batting.rename(columns = {'playerID':'Player ID',
                                    'yearID':'Year',
                                    'teamID':'Team',
                                    'G':'Games'})
print(batting.columns)
#batting = batting.set_index('Player ID')

Index(['Player ID', 'Year', 'Team', 'Games', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH', 'SF', 'GIDP'],
      dtype='object')


In [15]:
batting_final = pd.merge(batting_names[['IDPLAYER','PLAYERNAME', 'BATS']], batting,  right_on='Player ID', left_on='IDPLAYER')
batting_final = batting_final.rename(columns = {'PLAYERNAME':'Name', 'BATS':'Batting Stance'})

righties = 0
lefties = 0
switchies = 0
for i in batting_final.index:
    if (batting_final.loc[i, 'Batting Stance'] == 'R') & (batting_final.loc[i, 'Games'] >= 140):
        righties = righties + 1
    elif (batting_final.loc[i, 'Batting Stance'] == 'L') & (batting_final.loc[i, 'Games'] >= 140):
        lefties = lefties + 1
    elif (batting_final.loc[i, 'Batting Stance'] == 'B') & (batting_final.loc[i, 'Games'] >= 140):
        switchies = switchies + 1
print(lefties, righties, switchies)


238 316 67


In [23]:
px.histogram(batting_final, x='Batting Stance', title='Number of Players with Batting Stance In 2010-2015', labels={'count':'Number of Players'}).show()
px.histogram(batting_final[(batting_final['Batting Stance'] != 'B') & (batting_final['Games'] >= 0)], x='Batting Stance', y='H').show()
px.histogram(batting_final[(batting_final['Batting Stance'] != 'B') & (batting_final['Games'] >= 145)], x='Batting Stance', title='Players Who Played At Least 145 Games In A Season (2010-2015)').show()

In [ ]:
display(batting_final)
display(exp_stats)

,IDPLAYER,Name,Batting Stance,Player ID,Year,Team,Games,AB,R,H,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,aardsda01,David Aardsma,R,aardsda01,2010,SEA,53,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,aardsda01,David Aardsma,R,aardsda01,2012,NYA,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,aardsda01,David Aardsma,R,aardsda01,2013,NYN,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,aardsda01,David Aardsma,R,aardsda01,2015,ATL,33,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,abadfe01,Fernando Abad,L,abadfe01,2010,HOU,22,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6830,zobribe01,Ben Zobrist,B,zobribe01,2015,KCA,59,232.0,37.0,66.0,...,23.0,2.0,3.0,29.0,30.0,1.0,1.0,0.0,2.0,3.0
6831,zuninmi01,Mike Zunino,R,zuninmi01,2013,SEA,52,173.0,22.0,37.0,...,14.0,1.0,0.0,16.0,49.0,0.0,3.0,0.0,1.0,5.0
6832,zuninmi01,Mike Zunino,R,zuninmi01,2014,SEA,131,438.0,51.0,87.0,...,60.0,0.0,3.0,17.0,158.0,1.0,17.0,0.0,4.0,12.0
6833,zuninmi01,Mike Zunino,R,zuninmi01,2015,SEA,112,350.0,28.0,61.0,...,28.0,0.0,1.0,21.0,132.0,0.0,5.0,8.0,2.0,6.0


,Name,ID,Plate Appearances,Balls In Play,BA,xBA,xBA - BA Diff,Slg,xSlg,xSlg - Slg Diff,WOBA,xWOBA,xWOBA - WOBA Diff
0,Manny Machado,592518,713,528,0.286,0.262,0.024,0.502,0.447,0.055,0.370,0.345,0.025
1,Josh Donaldson,518626,711,499,0.297,0.280,0.017,0.568,0.546,0.022,0.398,0.390,0.008
2,Brian Dozier,572821,704,488,0.236,0.220,0.016,0.444,0.380,0.064,0.323,0.299,0.024
3,Anthony Rizzo,519203,701,488,0.278,0.275,0.003,0.512,0.496,0.016,0.384,0.383,0.001
4,Joey Votto,458015,695,412,0.314,0.289,0.025,0.541,0.541,0.000,0.427,0.422,0.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Aaron Hill,431094,353,267,0.230,0.237,-0.007,0.345,0.354,-0.009,0.281,0.296,-0.015
246,Clint Robinson,519208,352,258,0.272,0.287,-0.015,0.424,0.456,-0.032,0.340,0.362,-0.022
247,Randal Grichuk,545341,350,214,0.276,0.243,0.033,0.548,0.488,0.060,0.370,0.338,0.032
248,Cory Spangenberg,605486,344,239,0.271,0.253,0.018,0.399,0.346,0.053,0.319,0.298,0.021


In [ ]:
#print( statsapi.standings(leagueId=103,date='05/01/2023') )
#print( statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players',{'season':2018,'gameType':'W'})['people'] if x['fullName']=='Juan Soto'), 'hitting', 'season') )
players = exp_stats['Name']
# for data_name in exp_stats['Name']:
#     players.append(data_name)
print(players)
# with open('test_current_players.txt', 'w') as f:
#     for player in players:
#         try:
#             f.write(statsapi.player_stats(next(x['id'] for x in statsapi.get('sports_players',{'season':2018,'gameType':'W'})['people'] if x['fullName']==player), 'hitting', 'season'))
#         except:
#             print("Player Name not found in MLB Stats API")


0         Manny Machado
1        Josh Donaldson
2          Brian Dozier
3         Anthony Rizzo
4            Joey Votto
             ...       
245          Aaron Hill
246      Clint Robinson
247      Randal Grichuk
248    Cory Spangenberg
249      Jeff Francoeur
Name: Name, Length: 250, dtype: object


In [ ]:
# do BB, SO, and HBP
def hits_r(data):
    hits = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            hits += data.loc[i,'H']
    return hits
def players_r(data):
    players = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            players = players + 1
    return players
def games_r(data):
    games = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            games += data.loc[i, 'Games']
    return games
def runs_r(data):
    runs = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            runs += data.loc[i, 'R']
    return runs
def rbi_r(data):
    rbi = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            rbi += data.loc[i, 'RBI']
    return rbi
def sb_r(data):
    sb = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            sb += data.loc[i, 'SB']
    return sb
def bb_r(data):
    bb =0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            bb += data.loc[i, 'BB']
    return bb
def so_r(data):
    so = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            so += data.loc[i, 'SO']
    return so
def hbp_r(data):
    hbp = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'R':
            hbp += data.loc[i, 'HBP']
    return hbp

# do RBI, SB, BB, SO, and HBP
def hits_l(data):
    hits = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            hits += data.loc[i,'H']
    return hits
def players_l(data):
    players = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            players = players + 1
    return players
def games_l(data):
    games = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            games += data.loc[i, 'Games']
    return games
def runs_l(data):
    runs = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            runs += data.loc[i, 'R']
    return runs
def rbi_l(data):
    rbi = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            rbi += data.loc[i, 'RBI']
    return rbi
def sb_l(data):
    sb = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            sb += data.loc[i, 'SB']
    return sb
def bb_l(data):
    bb =0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            bb += data.loc[i, 'BB']
    return bb
def so_l(data):
    so = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            so += data.loc[i, 'SO']
    return so
def hbp_l(data):
    hbp = 0
    for i in data.index:
        if data.loc[i, 'Batting Stance'] == 'L':
            hbp += data.loc[i, 'HBP']
    return hbp



r_hits = batting_final.groupby('Year').apply(hits_r)
r_players = batting_final.groupby('Year').apply(players_r)
r_games = batting_final.groupby('Year').apply(games_r)
r_runs = batting_final.groupby('Year').apply(runs_r)
r_rbi = batting_final.groupby('Year').apply(rbi_r)
r_sb = batting_final.groupby('Year').apply(sb_r)
r_bb = batting_final.groupby('Year').apply(bb_r)
r_so = batting_final.groupby('Year').apply(so_r)
r_hbp = batting_final.groupby('Year').apply(hbp_r)

l_hits = batting_final.groupby('Year').apply(hits_l)
l_players = batting_final.groupby('Year').apply(players_l)
l_games = batting_final.groupby('Year').apply(games_l)
l_runs = batting_final.groupby('Year').apply(runs_l)
l_rbi = batting_final.groupby('Year').apply(rbi_l)
l_sb = batting_final.groupby('Year').apply(sb_l)
l_bb = batting_final.groupby('Year').apply(bb_l)
l_so = batting_final.groupby('Year').apply(so_l)
l_hbp = batting_final.groupby('Year').apply(hbp_l)

# do RBI, SB, BB, SO, and HBP
px.line(r_hits, title='Total Hits for Righties From 2010-2015', labels={'value':'Hits'}).show()
px.line(r_players, title='Total Right Handed Players From 2010-2015', labels={'value':'Players'}).show()
px.line(r_games, title='Total Games Played By Right Handers From 2010-2015', labels={'value':'Games'}).show()
px.line(r_runs, title='Total Runs Scored By Right Handers From 2010-2015', labels={'value':'Runs'}).show()
px.line(r_rbi, title='Total Runs Batted In By Right Handers From 2010-2015', labels={'value':'RBI'}).show()
px.line(r_sb, title='Total Stolen Bases By Right Handers From 2010-2015', labels={'value':'Stolen Bases'}).show()
px.line(r_bb, title='Total Walks By Right Handers From 2010-2015', labels={'value':'Walks'}).show()
px.line(r_so, title='Total Strikeouts By Right Handers From 2010-2015', labels={'value':'Strikeouts'}).show()
px.line(r_hbp, title='Total Right Handers Hit By Pitch From 2010-2015', labels={'value':'Right Handers Hit by Pitches'}).show()

# do RBI, SB, BB, SO, and HBP
px.line(l_hits, title='Total Hits for Lefties From 2010-2015', labels={'value':'Hits'}).show()
px.line(l_players, title='Total Left Handed Players From 2010-2015', labels={'value':'Players'}).show()
px.line(l_games, title='Total Games Played By Left Handers From 2010-2015', labels={'value':'Games'}).show()
px.line(l_runs, title='Total Runs Scored By Left Handers From 2010-2015', labels={'value':'Runs'}).show()
px.line(l_rbi, title='Total Runs Batted In By Left Handers From 2010-2015', labels={'value':'RBI'}).show()
px.line(l_sb, title='Total Stolen Bases By Left Handers From 2010-2015', labels={'value':'Stolen Bases'}).show()
px.line(l_bb, title='Total Walks By Left Handers From 2010-2015', labels={'value':'Walks'}).show()
px.line(l_so, title='Total Strikeouts By Left Handers From 2010-2015', labels={'value':'Strikeouts'}).show()
px.line(l_hbp, title='Total Left Handers Hit By Pitch From 2010-2015', labels={'value':'Left Handers Hit by Pitches'}).show()


